In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Metricas
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
# Dataset desbalanceado
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
# Modelos
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score, KFold
from Utils import *

In [3]:
df=pd.read_csv('/content/Data_Clean.csv')

Note que el dataset contiene múltiple variables categóricas, por lo que un one-hot-encoding es preferible, además de que las dimensiones extra podrían facilitar el empleo del Kernel en los modelos por venir.

In [4]:
df=pd.get_dummies(df)

In [5]:
df

,Unnamed: 0,Time,Amount,Age,Fraud,Day of Week_Tuesday,Day of Week_Wednesday,Type of Card_MasterCard,Type of Card_Visa,Entry Mode_CVC,...,Country of Residence_United Kingdom,Gender_F,Gender_M,Bank_Barclays,Bank_HSBC,Bank_Halifax,Bank_Lloyds,Bank_Metro,Bank_Monzo,Bank_RBS
0,0,19,5.0,25.2,False,0,1,0,1,0,...,1,0,1,0,0,0,0,0,0,1
1,1,17,288.0,49.6,False,0,1,1,0,0,...,0,1,0,0,0,0,1,0,0,0
2,2,14,5.0,42.2,False,0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
3,3,14,28.0,51.0,False,1,0,0,1,0,...,1,1,0,1,0,0,0,0,0,0
4,4,23,91.0,38.0,True,1,0,0,1,1,...,1,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99946,99946,22,15.0,53.8,False,1,0,1,0,0,...,1,1,0,0,0,1,0,0,0,0
99947,99947,23,7.0,45.0,False,0,1,1,0,0,...,0,0,1,1,0,0,0,0,0,0
99948,99948,11,21.0,46.5,False,1,0,1,0,0,...,1,1,0,0,1,0,0,0,0,0
99949,99949,22,25.0,48.2,False,0,1,0,1,0,...,1,0,1,1,0,0,0,0,0,0


In [6]:
df=df.drop('Unnamed: 0', axis=1)

Como mencionado en la clase de Redes Neuronales, es recomendable normalizar los datos, en especial tras hacer el one-hot-encoding siendo que las variables numéricas del data set podrían ser muy grandes y podrían complicar al modelo.

El scaling usual es el StandardScaler(); sin embargo, al no querer lidiar con valores negativos, se opta por el MinMaxScaler()

In [7]:
df.dtypes

Time                                       int64
Amount                                   float64
Age                                      float64
Fraud                                       bool
Day of Week_Tuesday                        uint8
Day of Week_Wednesday                      uint8
Type of Card_MasterCard                    uint8
Type of Card_Visa                          uint8
Entry Mode_CVC                             uint8
Entry Mode_PIN                             uint8
Entry Mode_Tap                             uint8
Type of Transaction_ATM                    uint8
Type of Transaction_Online                 uint8
Type of Transaction_POS                    uint8
Merchant Group_Children                    uint8
Merchant Group_Electronics                 uint8
Merchant Group_Entertainment               uint8
Merchant Group_Fashion                     uint8
Merchant Group_Food                        uint8
Merchant Group_Gaming                      uint8
Merchant Group_Produ

In [8]:
min_max_scaler=MinMaxScaler()
scaling = df[['Time','Amount','Age']]
scaling = pd.DataFrame(min_max_scaler.fit_transform(scaling), columns=scaling.columns)
scaling

,Time,Amount,Age
0,0.791667,0.000000,0.143460
1,0.708333,0.716456,0.486639
2,0.583333,0.000000,0.382560
3,0.583333,0.058228,0.506329
4,0.958333,0.217722,0.323488
...,...,...,...
99946,0.916667,0.025316,0.545710
99947,0.958333,0.005063,0.421941
99948,0.458333,0.040506,0.443038
99949,0.916667,0.050633,0.466948


In [9]:
df['Time']=scaling['Time']
df['Amount']=scaling['Amount']
df['Age']=scaling['Age']

Hecho el scaling y one-hot-encoding, podemos definir las variables X y Y para el resto del código.

In [10]:
X=df.drop('Fraud', axis=1)
Y=df['Fraud']

Debido al desbalanceo de datos antes señalado, se requiere emplear la librería *imbalanced-learn*.
Se emplearan 4 métodos:


*   El dataset sin modificaciones
*   Oversampling (Naive approach)
* Under sampling (Naive approach)
* SMOTEENN (una combinación de ambas)



In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [12]:
ros=RandomOverSampler(random_state=42)
X_oversampler, y_oversampler = ros.fit_resample(X, Y)
X_trainOver, X_testOver, y_trainOver, y_testOver = train_test_split(X_oversampler, y_oversampler, test_size=0.2)


In [13]:
rus = RandomUnderSampler(random_state=42)
X_undersampler, y_undersampler = rus.fit_resample(X, Y)
X_train_Under, X_test_Under, y_train_Under, y_test_Under = train_test_split(X_undersampler, y_undersampler, test_size=0.2)


Se empleará el usual KFolding de 10 folds para todo el código

In [14]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)


En el random forest, se debe decidir el número de árboles, por lo que se variará dicho parámetro

In [19]:
# DataSet desbalanceado
for i in (10,50,100,500):
  print('Number of trees: ', i)
  clf = RFC(n_estimators=i,random_state=0)
  y_pred,score,cnf_matrix=fit_predict_score_matrix(clf, X_train, y_train, X_test, y_test)
  # Obtenermos matrix de confusión
  print(cnf_matrix)
  # Imprimimos los scores
  tn, fp, fn, tp = cnf_matrix.ravel()
  precision = metrics.precision_score(y_test, y_pred)
  exact=metrics.accuracy_score(y_test, y_pred)
  sensitivity = metrics.recall_score(y_test, y_pred)
  especif=tn / (tn + fp)
  print("Precision:", precision)
  print("Exactitud:", exact)
  print("Sensibilidad:", sensitivity)
  print("Especificidad:", especif)
  cross_val_exactitud = cross_val_score(clf, X_test, y_test, cv=kf)
  cross_val_precision = cross_val_score(clf, X_test, y_test, cv=kf, scoring='precision')
  print("Exactitud promedio:", np.mean(cross_val_exactitud))
  print("Precisión promedio:", np.mean(cross_val_precision))
  print('############# New number of trees###############')

Number of trees:  10
Score sin validación cruzada:  0.985843629633335
[[18529    37]
 [  246  1179]]
Precision: 0.9695723684210527
Exactitud: 0.985843629633335
Sensibilidad: 0.8273684210526315
Especificidad: 0.9980071097705483
Exactitud promedio: 0.9811916208104051
Precisión promedio: 0.9446228268325007
############# New number of trees###############
Number of trees:  50
Score sin validación cruzada:  0.9867940573257966
[[18528    38]
 [  226  1199]]
Precision: 0.9692805173807599
Exactitud: 0.9867940573257966
Sensibilidad: 0.8414035087719298
Especificidad: 0.997953247872455
Exactitud promedio: 0.9841429464732366
Precisión promedio: 0.9611393231728027
############# New number of trees###############
Number of trees:  100
Score sin validación cruzada:  0.9871942374068331
[[18532    34]
 [  222  1203]]
Precision: 0.9725141471301536
Exactitud: 0.9871942374068331
Sensibilidad: 0.8442105263157895
Especificidad: 0.9981686954648282
Exactitud promedio: 0.9844930715357678
Precisión promedio: 0.

In [20]:
# DataSet undersampling
for i in (10,50,100,500):
  clf = RFC(n_estimators=i,random_state=0)
  print('Number of trees: ', i)
  y_pred_Under,score,cnf_matrix=fit_predict_score_matrix(clf, X_train_Under, y_train_Under, X_test_Under, y_test_Under)
  # Obtenermos matrix de confusión
  print(cnf_matrix)
  # Imprimimos los scores
  tn, fp, fn, tp = cnf_matrix.ravel()
  precision = metrics.precision_score(y_test_Under, y_pred_Under)
  exact=metrics.accuracy_score(y_test_Under, y_pred_Under)
  sensitivity = metrics.recall_score(y_test_Under, y_pred_Under)
  especif=tn / (tn + fp)
  print("Precision:", precision)
  print("Exactitud:", exact)
  print("Sensibilidad:", sensitivity)
  print("Especificidad:", especif)
  cross_val_exactitud = cross_val_score(clf, X_test_Under, y_test_Under, cv=kf)
  cross_val_precision = cross_val_score(clf, X_test_Under, y_test_Under, cv=kf, scoring='precision')
  print("Exactitud promedio:", np.mean(cross_val_exactitud))
  print("Precisión promedio:", np.mean(cross_val_precision))
  print('############# New number of trees###############')

Number of trees:  10
Score sin validación cruzada:  0.9523643949930459
[[1363   69]
 [  68 1376]]
Precision: 0.9522491349480969
Exactitud: 0.9523643949930459
Sensibilidad: 0.9529085872576177
Especificidad: 0.9518156424581006
Exactitud promedio: 0.9485324719318621
Precisión promedio: 0.9370652085616952
############# New number of trees###############
Number of trees:  50
Score sin validación cruzada:  0.9558414464534075
[[1351   81]
 [  46 1398]]
Precision: 0.9452332657200812
Exactitud: 0.9558414464534075
Sensibilidad: 0.9681440443213296
Especificidad: 0.9434357541899442
Exactitud promedio: 0.9530499903213316
Precisión promedio: 0.9328548294550723
############# New number of trees###############
Number of trees:  100
Score sin validación cruzada:  0.9551460361613352
[[1349   83]
 [  46 1398]]
Precision: 0.9439567859554355
Exactitud: 0.9551460361613352
Sensibilidad: 0.9681440443213296
Especificidad: 0.9420391061452514
Exactitud promedio: 0.9533996322106078
Precisión promedio: 0.931891385

In [23]:
# DataSet Oversampling
for i in (10,50,100,500):
  clf = RFC(n_estimators=i,random_state=0)
  print('Number of trees: ', i)
  y_predOver,score,cnf_matrix=fit_predict_score_matrix(clf, X_trainOver, y_trainOver, X_testOver, y_testOver)
  # Obtenermos matrix de confusión
  print(cnf_matrix)
  # Imprimimos los scores
  tn, fp, fn, tp = cnf_matrix.ravel()
  precision = metrics.precision_score(y_testOver, y_predOver)
  exact=metrics.accuracy_score(y_testOver, y_predOver)
  sensitivity = metrics.recall_score(y_testOver, y_predOver)
  especif=tn / (tn + fp)
  print("Precision:", precision)
  print("Exactitud:", exact)
  print("Sensibilidad:", sensitivity)
  print("Especificidad:", especif)
  cross_val_exactitud = cross_val_score(clf, X_testOver, y_testOver, cv=kf)
  cross_val_precision = cross_val_score(clf, X_testOver, y_testOver, cv=kf, scoring='precision')
  print("Exactitud promedio:", np.mean(cross_val_exactitud))
  print("Precisión promedio:", np.mean(cross_val_precision))
  print('############# New number of trees###############')

Number of trees:  10
Score sin validación cruzada:  0.9963077752324485
[[18433   137]
 [    0 18535]]
Precision: 0.9926628106255355
Exactitud: 0.9963077752324485
Sensibilidad: 1.0
Especificidad: 0.9926225094238018
Exactitud promedio: 0.9771458641570445
Precisión promedio: 0.9640437228347889
############# New number of trees###############
Number of trees:  50
Score sin validación cruzada:  0.9966311817814311
[[18445   125]
 [    0 18535]]
Precision: 0.9933011789924974
Exactitud: 0.9966311817814311
Sensibilidad: 1.0
Especificidad: 0.9932687129779214
Exactitud promedio: 0.9749627210137269
Precisión promedio: 0.957675727990412
############# New number of trees###############
Number of trees:  100
Score sin validación cruzada:  0.9966042312356825
[[18444   126]
 [    0 18535]]
Precision: 0.9932479502706179
Exactitud: 0.9966042312356825
Sensibilidad: 1.0
Especificidad: 0.9932148626817447
Exactitud promedio: 0.9750974701132569
Precisión promedio: 0.9577768722098277
############# New number o

En cuanto al Vector Support Machine (VSM), aprovechando la gran dimensionalidad del dataset, podemos emplear el kernel Radial basis function (rbf), y variar el parámetro de regularización.

In [25]:
# DataSet desbalanceado
for i in (0.01,0.1,1,10,100):
  print('Value of C: ', i)
  clf = SVC(C=i,random_state=0, kernel='rbf')
  y_pred,score,cnf_matrix=fit_predict_score_matrix(clf, X_train, y_train, X_test, y_test)
  # Obtenermos matrix de confusión
  print(cnf_matrix)
  # Imprimimos los scores
  # tn, fp, fn, tp = cnf_matrix.ravel()
  # precision = metrics.precision_score(y_test, y_pred)
  # exact=metrics.accuracy_score(y_test, y_pred)
  # sensitivity = metrics.recall_score(y_test, y_pred)
  # especif=tn / (tn + fp)
  # print("Precision:", precision)
  # print("Exactitud:", exact)
  # print("Sensibilidad:", sensitivity)
  # print("Especificidad:", especif)
  # cross_val_exactitud = cross_val_score(clf, X_test, y_test, cv=kf)
  # cross_val_precision = cross_val_score(clf, X_test, y_test, cv=kf, scoring='precision')
  # print("Exactitud promedio:", np.mean(cross_val_exactitud))
  # print("Precisión promedio:", np.mean(cross_val_precision))
  print('############# New value of C ###############')

Value of C:  0.01
Score sin validación cruzada:  0.9400230103546596
[[18565     1]
 [ 1198   227]]
############# New value of C ###############
Value of C:  0.1
Score sin validación cruzada:  0.9693362012905807
[[18394   172]
 [  441   984]]
############# New value of C ###############
Value of C:  1
Score sin validación cruzada:  0.9772897804011805
[[18429   137]
 [  317  1108]]
############# New value of C ###############
Value of C:  10
Score sin validación cruzada:  0.9826922114951728
[[18452   114]
 [  232  1193]]
############# New value of C ###############
Value of C:  100
Score sin validación cruzada:  0.9769896453404032
[[18350   216]
 [  244  1181]]
############# New value of C ###############


# Resultados
Los modelos se desempeñaron en una variedad de data sets


*   Sin modificaciones: Los puntajes favorecen ligeramente al bosque de 100 árboles, aunque la moda está cerca de 95% para el resto de los puntajes y bosques, excepto para la sensibilidad, lo cual muestra tendencia a clasificar falsos positivos.
*   El undersampling mejoró los puntajes en general, no bajando del 93% , se encuentra que el bosque de 50 árboles es ligeramente superior al resto.
* En cuanto al undersampling, no sólo todas las medidas incrementaron, si no que se llegó a una sensibilidad perfecta, lo cual indica que el modelo ya no tiende a clasificar incorrectamente falsos positivos.
* Finalmente, a manera de experimentación, se empleo un Support Vector Machine con kernel rbf, dado que es el más preciso para la definición de boundaries. Notesé que, a pesar de variar el parámetro de regularización, el modelo es notoriamente más débil que cualquiera de los random forests.



